In [1]:
# %pip install tcrdist3
import tcrdist

In [2]:
from matplotlib import pyplot as plt
from eindex.numpy import argsort, gather_scatter
import pandas as pd
import numpy as np

In [3]:
if False:
    vdj_df = pd.read_csv('./VDJdb_paired_chain.csv')
    is_good = ~(vdj_df['CDR3a_extended'].isna() | vdj_df['CDR3b_extended'].isna())
    paired_TRA = vdj_df['CDR3a_extended'][is_good].tolist()
    paired_TRB = vdj_df['CDR3b_extended'][is_good].tolist()
    
    pep_lookup, paired_pep = np.unique(vdj_df['Peptide'][is_good], return_inverse=True)
    peptide_ids = paired_pep
else:
    vdj_df = pd.read_csv('/Users/axelr/Downloads/vdjdb.tsv', sep='\t')

    paired_TRA = []
    paired_TRB = []
    peptides = []
    v_a_gene = []
    v_b_gene = []
    for _, group in vdj_df[vdj_df['complex.id'] != 0].groupby('complex.id'):
        # print(group)
        if len(group) != 2:
            continue
        if group['Score'].iloc[0] == 0:
            continue
    
        group = group.sort_values(by='Gene')
        _tra, _trb = group['CDR3']
    
        paired_TRA.append(_tra)
        paired_TRB.append(_trb)
        v_a_gene.append(group['V'].iloc[0])
        v_b_gene.append(group['V'].iloc[1])
        peptides.append(group['Epitope'].iloc[0])
        
    pep_lookup, paired_pep = np.unique(peptides, return_inverse=True)
    peptide_ids = paired_pep

    df_tcr = pd.DataFrame({
        'cdr3_a_aa': paired_TRA,
        'cdr3_b_aa': paired_TRB,
        'v_a_gene': v_a_gene,
        'v_b_gene': v_b_gene,
        'peptide': peptides,
        'peptide_id': peptide_ids,
    })
    df_tcr = df_tcr.drop_duplicates()

In [4]:
_, counts = np.unique(peptides, return_counts=True)

In [5]:
from collections import Counter

if False:
    # kmer stats
    def kmers(seq: str, k=3) -> list[str]:
        return [seq[i:i+k] for i in range(len(seq) - k + 1)]
    
    def count_kmers(seq: list[str], k) -> dict[str, int]:
        counter = Counter()
        for s in seq:
            for kmer in kmers(s, k):
                counter[kmer] += 1
        return counter

    for k in [2, 3, 4, 5]:
        counts = count_kmers(peptides, k)
        print()
        for minimal in [1, 2, 3, 4, 5]:
            print(k, minimal, sum(v >= minimal for k, v in counts.items()))

## TCR dist

In [6]:
df_tcr['v_a_gene'].sort_values().unique()

array(['TRAV1*01', 'TRAV1-1*01', 'TRAV1-2*01', 'TRAV10*01', 'TRAV10N*01',
       'TRAV11*02', 'TRAV12-1*01', 'TRAV12-2*01', 'TRAV12-3*01',
       'TRAV12D-1*01', 'TRAV12N-2*01', 'TRAV12N-3*01', 'TRAV13-1*01',
       'TRAV13-2*01', 'TRAV13-5*01', 'TRAV13N-2*01', 'TRAV14-1*01',
       'TRAV14-2*02', 'TRAV14-3*01', 'TRAV14/DV4*01', 'TRAV14/DV4*02',
       'TRAV14D-1*01', 'TRAV14D-2*01', 'TRAV14D-3/DV8*01',
       'TRAV14D-3/DV8*02', 'TRAV14D-3/DV8*08', 'TRAV14N-1*01',
       'TRAV14N-2*01', 'TRAV14N-3*01', 'TRAV15-1/DV6-1*01',
       'TRAV15N-1*01', 'TRAV16*01', 'TRAV16D/DV11*01', 'TRAV16D/DV11*02',
       'TRAV16N*01', 'TRAV17*01', 'TRAV18*01', 'TRAV19*01', 'TRAV2*01',
       'TRAV20*01', 'TRAV21*01', 'TRAV21/DV12*01', 'TRAV22*01',
       'TRAV23/DV6*01', 'TRAV24*01', 'TRAV25*01', 'TRAV26-1*01',
       'TRAV26-2*01', 'TRAV27*01', 'TRAV29/DV5*01', 'TRAV3*01',
       'TRAV3-1*01', 'TRAV3-3*01', 'TRAV3-4*01', 'TRAV30*01', 'TRAV34*01',
       'TRAV35*01', 'TRAV36/DV7*01', 'TRAV38-1*01', 'TRA

In [7]:
if True:
    pass
else:
    vdj_df = pd.read_csv('VDJdb_paired_chain.csv', sep='\t')
    # is_good = ~(vdj_df['CDR3a_extended'].isna() | vdj_df['CDR3b_extended'].isna())    
    
    # drop duplicates
    _, peptide_ids = np.unique(df['Peptide'], return_inverse=True)    
    df_tcr = df.rename(columns={
        'CDR3a': 'cdr3_a_aa',
        'CDR3b': 'cdr3_b_aa',
        'Va': 'v_a_gene',
        'Vb': 'v_b_gene',
    })

In [8]:
df_tcr

,cdr3_a_aa,cdr3_b_aa,v_a_gene,v_b_gene,peptide,peptide_id
0,CIVRAPGRADMRF,CASSYLPGQGDHYSNQPQHF,TRAV26-1*01,TRBV13*01,FLKEKGGL,68
1,CAVPSGAGSYQLTF,CASSFEPGQGFYSNQPQHF,TRAV20*01,TRBV13*01,FLKEKGGL,68
2,CAVKASGSRLT,CASSYEPGQVSHYSNQPQHF,TRAV2*01,TRBV13*01,FLKEKGGL,68
3,CAYRPPGTYKYIF,CASSALASLNEQFF,TRAV38-2/DV8*01,TRBV14*01,FLKEKGGL,68
4,CIVRAPGRADMRF,CASSYLPGQGDHYSNQPQHF,TRAV26-1*01,TRBV13*01,FLKEQGGL,70
...,...,...,...,...,...,...
2820,CIALNARLMF,CASSLRATDTQYF,TRAV26-1*01,TRBV7-2*01,PQPELPYPQPQL,233
2821,CAMREGRYSSASKIIF,CATSRAGGGGEKLFF,TRAV14/DV4*01,TRBV15*01,FPQPEQPFPWQP,79
2822,CLVGDGDGGATNKLIF,CASSQGSGGNEQFF,TRAV4*01,TRBV4-3*01,FPQPEQPFPWQP,79
2823,CAASVLYGSSNTGKLIF,CASSIVGSGGYNEQFF,TRAV29/DV5*01,TRBV19*01,QLQPFPQPELPY,242


In [9]:
from tcrdist.repertoire import TCRrep
# tcrdist complains a lot, and drops quite a lot of data. I just ignored that

rep = TCRrep(organism='human', cell_df=df_tcr)
rep.compute_distances()

/Users/axelr/pll/venv/lib/python3.10/site-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/Users/axelr/pll/venv/lib/python3.10/site-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")
/Users/axelr/pll/venv/lib/python3.10/site-packages/tcrdist/repertoire.py:500: UserWarning: TRAV14D-2*01 gene was not recognized in reference db no cdr seq could be inferred
  f0 = lambda v : self._map_gene_to_reference_seq2(gene = v,
/Users/axelr/pll/venv/lib/python3.10/site-packages/tcrdist/repertoire.py:500: UserWarning: TRAV9-4*01 gene was not recognized in reference db no cdr seq could be inferred
  f0 = lambda v : self._map_gene_to_reference_seq2(gene = v,
/Users/axelr/pll/venv/lib/python3.10/site-packages/tcrdist/repertoire.py:500: UserWarning: TRAV14-1*01 gene was not recognized in reference

In [10]:
rep.clone_df.shape

(1583, 14)

In [11]:
# see how much data was discarded
distances = rep.pw_cdr3_b_aa
distances.shape, df_tcr.shape

((1583, 1583), (2335, 7))

In [12]:
for dist in [
    rep.pw_cdr3_a_aa,
    rep.pw_cdr3_b_aa,
    rep.pw_cdr3_a_aa + rep.pw_cdr3_b_aa,
]:
    peptide_ids = rep.clone_df['peptide_id'].values
    arg = argsort(dist  + np.eye(len(dist)) * 10_000, 'i j -> [i] j order')[:, :, :3].copy()
    mask = gather_scatter(peptide_ids, arg, 'i, [i] j order -> j order') == peptide_ids[:, None]
    score = mask.mean(axis=1)
    print(score.mean())

0.2543693409138766
0.2695304274584123
0.3051168667087808
